# Experiments with Transformer-based models

Let us reproduce fine-tuning of T5 model as authors of given paper did in [their work](https://huggingface.co/s-nlp/t5-paranmt-detox). I am planning to try different T5 model sizes for fine-tuning. Inspired by [the fine-tuning Translation example from HuggingFace](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/translation.ipynb). 

In [11]:
import warnings
import numpy as np
import pandas as pd
import torch

# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '3'

# Set seeds
torch.manual_seed(42)
np.random.seed(42)
warnings.filterwarnings('ignore')

## Data preprocessing

In [15]:
from datasets import Dataset, DatasetDict
from transformers import T5Tokenizer


tokenizer_checkpoint_path = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(tokenizer_checkpoint_path)

train_df = pd.read_csv('../data/interim/train.csv', index_col=0)
val_df = pd.read_csv('../data/interim/val.csv', index_col=0)
test_df = pd.read_csv('../data/interim/test.csv', index_col=0)

In [16]:
prefix = "detoxify:"
max_input_length = 256
max_target_length = 256


def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples["reference"]]
    targets = [ex for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

### Build HuggingFace dataset

In [17]:
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test_df)
ds = DatasetDict()
ds['train'] = train_ds
ds['val'] = val_ds
ds['test'] = test_ds
ds = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/506998 [00:00<?, ? examples/s]

Map:   0%|          | 0/56334 [00:00<?, ? examples/s]

Map:   0%|          | 0/14445 [00:00<?, ? examples/s]

In [18]:
hf_ds_dir = '../data/interim/dshf'
ds.save_to_disk(hf_ds_dir)

Saving the dataset (0/1 shards):   0%|          | 0/506998 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/56334 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14445 [00:00<?, ? examples/s]

## Model training

In [13]:
from datasets import load_from_disk


hf_ds_dir = '../data/interim/dshf/'
ds = load_from_disk(hf_ds_dir)

In [26]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments
from transformers import T5Tokenizer, T5ForConditionalGeneration


model_name = 't5-base'
batch_size = 32
training_model_name = f"{model_name}-finetuned-paranmt500k-detox"

args = Seq2SeqTrainingArguments(
    output_dir=f'../models/{training_model_name}',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    report_to='wandb',
)

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [23]:
from datasets import load_metric


metric = load_metric("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
from transformers import Seq2SeqTrainer


trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=ds['train'],
    eval_dataset=ds['val'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [25]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3'

In [ ]:
trainer.train()

## Model inference